In [1]:
!pip install urduhack
#!pip install tensorflow

In [2]:
import urduhack
urduhack.download()

2024-06-07 15:16:17.204117: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 15:16:17.204176: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 15:16:17.206068: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our Tens

In [4]:
import pandas as pd
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Embedding, Reshape, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



# TextGAN

In [5]:
#libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, LSTM, Flatten
# using textgan
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, Flatten
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urduhack
from urduhack.preprocessing import normalize_whitespace, remove_punctuation
from urduhack.tokenization import word_tokenizer
from urduhack.normalization import normalize


In [6]:
# Assuming you have functions normalize, normalize_whitespace, remove_punctuation, word_tokenizer defined
def preprocess_text(text):
    text = normalize(text)
    text = normalize_whitespace(text)
    text = remove_punctuation(text)
    tokens = word_tokenizer(text)
    return tokens


In [7]:
# data loading 
dataset_path = "/kaggle/input/urdu-news-csv/news.csv"

df = pd.read_csv(dataset_path, encoding='utf-8')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
#removing space in label columns 
df['label'] = df['label'].str.strip()
org_data = df.copy()
# Verify the unique labels to ensure they are cleaned
print("Unique labels after cleaning:", org_data['label'].unique())

print(df.shape)
df.head()

Unique labels after cleaning: ['real' 'fake']
(4097, 2)


,news,label
0,\tپاکستان کے صوبے پنجاب میں گذشتہ ہفتے ہونے وا...,real
1,\tوزیر اعظم عمران خان کے ساجد گوندل کی بازیابی...,real
2,لاہور(کلچرل رپورٹر)ایشین ایکسی لینس پرفارمنس ا...,fake
3,امریکی خواتین سوکر اسٹار میگن ریپینو گھٹنوں کے...,fake
4,\tوفاق ہو یا صوبے، خیبر پختونخواہ ہو یا پنجاب،...,real


In [8]:
# Pre Processing using urduhack functions
df = df.head(1000)  # Limit to the top 1000 rows due to resource limitations you can increase
df['label'] = df['label'].str.strip()
df['tokens'] = df['news'].apply(preprocess_text)


7/7 [==============================] - 1s 131ms/step


In [9]:
df.head() # latest data set wit tokens 

,news,label,tokens
0,\tپاکستان کے صوبے پنجاب میں گذشتہ ہفتے ہونے وا...,real,"[پاکستان, کے, صوبے, پنجاب, میں, گذشتہ, ہفتے, ہ..."
1,\tوزیر اعظم عمران خان کے ساجد گوندل کی بازیابی...,real,"[وزیراعظم, عمران, خان, کے, ساجد, گوندل, کی, با..."
2,لاہور(کلچرل رپورٹر)ایشین ایکسی لینس پرفارمنس ا...,fake,"[لاہور, کلچرل, رپورٹر, ایشین, ایکسیلینس, پرفار..."
3,امریکی خواتین سوکر اسٹار میگن ریپینو گھٹنوں کے...,fake,"[امریکی, خواتین, سو, کر, اسٹار, میگن, ریپینو, ..."
4,\tوفاق ہو یا صوبے، خیبر پختونخواہ ہو یا پنجاب،...,real,"[وفاق, ہو, یا, صوبے, خیبر, پختونخواہ, ہو, یا, ..."


In [10]:
# tokeniazation , pad sequencing 
all_tokens = [token for sublist in df['tokens'] for token in sublist]
vocab = list(set(all_tokens))
token_to_id = {token: idx for idx, token in enumerate(vocab)}
df['token_ids'] = df['tokens'].apply(lambda tokens: [token_to_id[token] for token in tokens])
max_sequence_length = max([len(token_ids) for token_ids in df['token_ids']])
padded_sequences = pad_sequences(df['token_ids'], maxlen=max_sequence_length, padding='post')


In [11]:
# Preparing data for  GAN 
data = pd.DataFrame(padded_sequences)
data['label'] = df['label'].apply(lambda x: 1 if x == 'real' else 0)
data['label'] = data['label'].astype(int)

for column in data.columns:
    if column != 'label':
        data[column] = data[column].astype(float)

data.columns = data.columns.astype(str)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
data_tensor = tf.convert_to_tensor(data_scaled, dtype=tf.float32)
label_tensor = tf.convert_to_tensor(data['label'].values, dtype=tf.float32)
dataset = tf.data.Dataset.from_tensor_slices((data_tensor, label_tensor)).shuffle(buffer_size=1024).batch(32)


In [12]:
# defoinning generator and discriminator yuo can enhance its complexity accoridng to need 
latent_dim = 100
output_dim = data_tensor.shape[1]

generator = Sequential([
    Dense(128, input_dim=latent_dim, activation='relu'),
    Reshape((1, 128)),
    LSTM(256, return_sequences=True),
    Flatten(),
    Dense(output_dim, activation='sigmoid')
])
input_dim = data_tensor.shape[1]

discriminator = Sequential([
    Reshape((1, input_dim), input_shape=(input_dim,)),
    LSTM(256, return_sequences=True),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [13]:
# Defining TextGAN
optimizer = tf.keras.optimizers.legacy.Adam(lr=0.0002)
discriminator.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
discriminator.trainable = False

gan_input = tf.keras.Input(shape=(latent_dim,))
generated_data = generator(gan_input)
gan_output = discriminator(generated_data)
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(optimizer=optimizer, loss='binary_crossentropy')



/opt/conda/lib/python3.10/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [14]:
# training GAN
epochs = 100

for epoch in range(epochs):
    for real_data, _ in dataset:
        batch_size = real_data.shape[0]

        # Train Discriminator
        real_labels = tf.ones((batch_size, 1))
        fake_labels = tf.zeros((batch_size, 1))

        z = tf.random.normal((batch_size, latent_dim))
        fake_data = generator(z)

        d_loss_real = discriminator.train_on_batch(real_data, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        z = tf.random.normal((batch_size, latent_dim))
        g_loss = gan.train_on_batch(z, real_labels)

    print(f'Epoch [{epoch+1}/{epochs}], d_loss: {d_loss[0]:.4f}, g_loss: {g_loss:.4f}')


Epoch [1/100], d_loss: 0.4163, g_loss: 1.3984
Epoch [2/100], d_loss: 0.5129, g_loss: 0.9110
Epoch [3/100], d_loss: 0.3634, g_loss: 1.0567
Epoch [4/100], d_loss: 0.3960, g_loss: 1.2414
Epoch [5/100], d_loss: 0.1704, g_loss: 1.4984
Epoch [6/100], d_loss: 0.0870, g_loss: 1.9914
Epoch [7/100], d_loss: 0.0877, g_loss: 1.8808
Epoch [8/100], d_loss: 0.1001, g_loss: 2.0762
Epoch [9/100], d_loss: 0.0734, g_loss: 2.1825
Epoch [10/100], d_loss: 0.1220, g_loss: 1.5567
Epoch [11/100], d_loss: 0.1124, g_loss: 1.6144
Epoch [12/100], d_loss: 0.0854, g_loss: 1.9366
Epoch [13/100], d_loss: 0.0618, g_loss: 2.2853
Epoch [14/100], d_loss: 0.0393, g_loss: 2.6071
Epoch [15/100], d_loss: 0.0290, g_loss: 2.9217
Epoch [16/100], d_loss: 0.0215, g_loss: 3.2020
Epoch [17/100], d_loss: 0.0169, g_loss: 3.4659
Epoch [18/100], d_loss: 0.0136, g_loss: 3.7038
Epoch [19/100], d_loss: 0.0103, g_loss: 3.9194
Epoch [20/100], d_loss: 0.0084, g_loss: 4.1175
Epoch [21/100], d_loss: 0.0069, g_loss: 4.2992
Epoch [22/100], d_loss

In [15]:
# generating text 
num_samples = 100

z = tf.random.normal((num_samples, latent_dim))
synthetic_data = generator(z).numpy()
synthetic_data = scaler.inverse_transform(synthetic_data)
print(synthetic_data)

[[4.2562324e+03 4.5407598e+03 4.4357690e+03 ... 6.1002660e-01
  5.8282232e+00 4.3800187e-01]
 [4.2562378e+03 4.5407617e+03 4.4357695e+03 ... 6.1002976e-01
  5.8282509e+00 4.3800202e-01]
 [4.2562378e+03 4.5407622e+03 4.4357695e+03 ... 6.1002994e-01
  5.8282580e+00 4.3800199e-01]
 ...
 [4.2562383e+03 4.5407607e+03 4.4357700e+03 ... 6.1002874e-01
  5.8282466e+00 4.3800205e-01]
 [4.2562422e+03 4.5407646e+03 4.4357705e+03 ... 6.1003417e-01
  5.8282814e+00 4.3800232e-01]
 [4.2562319e+03 4.5407607e+03 4.4357690e+03 ... 6.1002648e-01
  5.8282313e+00 4.3800193e-01]]


# Text Geenration 

In [16]:
# converting generated text into back  in urdu 
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming 'dataset' is already created from the code provided
data_tensor, label_tensor = zip(*[(data, label) for data, label in dataset])
data_array = tf.concat(data_tensor, axis=0).numpy()
label_array = tf.concat(label_tensor, axis=0).numpy()

# Convert back to DataFrame
data_df = pd.DataFrame(data_array)
data_df['label'] = label_array

# Separate features and labels
features = data_df.drop(columns='label')
labels = data_df['label']


# Apply inverse transform only to the feature columns
features_original = scaler.inverse_transform(features)

# Create a DataFrame for the original data
data_df = pd.DataFrame(features_original, columns=features.columns)
data_df['label'] = labels

# Reconstruct the token_to_id dictionary
all_tokens = [token for sublist in df['tokens'] for token in sublist]
vocab = list(set(all_tokens))
token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for token, idx in token_to_id.items()}

# Remove padding
def remove_padding(sequence, pad_value=0):
    return [int(token_id) for token_id in sequence if token_id != pad_value]

synthetic_sequences = [remove_padding(seq) for seq in data_df.drop(columns='label').values]
# Generate synthetic_labels (assuming the label column represents binary classification)
synthetic_labels = ["real" if label == 1 else "fake" for label in labels]

# Convert token IDs back to tokens
def ids_to_tokens(ids, id_to_token):
    return [id_to_token[int(id)] for id in ids if int(id) in id_to_token]

synthetic_texts = [ids_to_tokens(seq, id_to_token) for seq in synthetic_sequences]

# Join tokens to form text
def tokens_to_text(tokens):
    return ' '.join(tokens)

synthetic_texts = [tokens_to_text(tokens) for tokens in synthetic_texts]

# Display the generated texts
for i, text in enumerate(synthetic_texts):
    print(f"Synthetic Text {i+1}: {text}\n")


Synthetic Text 1: ایک پی اے سی جو اصولی طور پر قدامت پسند امیدواروں کی حمایت کرنے ڈریڈکو دعوی کرتیہے جو آئینی طور پر محدود حکومت کہناہے منڈیوں اور ہماری جمہوریہ کو انفرادی آزادی کی بحالی کی جدوجہد کی راہنمائی کرے گینے جارجیا میں ایک اشتہار جاری کیاہے جس میں ڈیموکریٹ حرکت آسف کو ہاؤس میجریٹیوہپ کی فائرنگ سے کمرے والا دکھائی دیتا ہے                         

Synthetic Text 2: یہ اس سے کہیں زیادہ خراب نہیں ہوتاہے یہ خیال کہ ہم اپنے تجربہ کاروں کو صحت کی بہترین دیکھ سعد دستیاب وکٹوں کر رہے ہیںوہ ایک پیلاسے باہرہے تبدیلیوں ایک زخمی تجربہ کاراور تینوں کااکیلے تباد جھلس رہاہےپاکستانی تبدیلیوں وہ بہادری سے دو فوجیوں کو بچانے کے عابد ٹرک کی آگ میں گھس گیا جس نے بالآخردم توڑدیااب وہ خط بیچ رہی ہیںجو ان ہیںصدر اوباما کی جانب سے                

Synthetic Text 3: اصطلاح کی حدود ملزوم کی حدود مدت کی حدود میکسین واٹرس مدت کی حدود کے عابد پنجے چائلڈہے وہ ایک گندی اور بے مظاہرین تلخ نسل پرست ہیںجنھیں ٹرمپ سے اتنی گہری بشمول ہے کہ وہ ان کے ساتھ کام کرنے سے ان کار کر دیتیہیںٹھیکہے اس کے ساتھ اچھی قسمت ہ

In [17]:
import pandas as pd

# Assuming synthetic_texts and synthetic_labels are already created
# Create a DataFrame for the synthetic texts and labels
synthetic_data_df = pd.DataFrame({
    'news': synthetic_texts,
    'label': synthetic_labels
})

# Combine the synthetic DataFrame with the original DataFrame
combined_df = pd.concat([org_data, synthetic_data_df], ignore_index=True)

# Display the combined DataFrame
print(combined_df.shape)
combined_df.head()


(5097, 2)


,news,label
0,\tپاکستان کے صوبے پنجاب میں گذشتہ ہفتے ہونے وا...,real
1,\tوزیر اعظم عمران خان کے ساجد گوندل کی بازیابی...,real
2,لاہور(کلچرل رپورٹر)ایشین ایکسی لینس پرفارمنس ا...,fake
3,امریکی خواتین سوکر اسٹار میگن ریپینو گھٹنوں کے...,fake
4,\tوفاق ہو یا صوبے، خیبر پختونخواہ ہو یا پنجاب،...,real


> as you can see the org data shap was (4097,2) and we train our GAN ith 1000 rows  , wwe have generated 1000 rows new so adding them in org data it becomes (5097,2)

# Applying Model 

> firstly applying adaboost so we have trioi convert combined_df back to tokenize and then we will apply adaboost

In [18]:
import pandas as pd
from urduhack.preprocessing import normalize_whitespace, remove_punctuation
from urduhack.tokenization import word_tokenizer
from urduhack.normalization import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Set a random seed for reproducibility
np.random.seed(42)  
import pandas as pd

# removing space in label columns 
combined_df['label'] = combined_df['label'].str.strip()

print("Total number of rows in the dataset:", len(combined_df))

# Shuffle the rows of the combined DataFrame
combined_df = combined_df.sample(frac=1).reset_index(drop=True)
# Limit the number of rows
num_rows = 500  # as we have limited resource so i choose top 500 ros after shuffling you can increase 
combined_df = combined_df.head(num_rows)



# Define the preprocess_text function
def preprocess_text(text):
    text = normalize(text)
    text = normalize_whitespace(text)
    text = remove_punctuation(text)
    tokens = word_tokenizer(text)
    return tokens

# Apply the preprocessing function to the dataframe
combined_df['tokens'] = combined_df['news'].apply(preprocess_text)

# Remove Urdu stop words
urdu_stop_words = ["کے", "میں", "کی", "کا", "ہے", "کو", "سے", "پر", "اور"]  # Example stop words

def remove_stop_words(tokens):
    filtered_tokens = [token for token in tokens if token not in urdu_stop_words]
    return ' '.join(filtered_tokens)

combined_df['cleaned_news'] = combined_df['tokens'].apply(remove_stop_words)

# Feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(combined_df['cleaned_news'])

# Label encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(combined_df['label'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)




Total number of rows in the dataset: 5097
8/8 [==============================] - 1s 117ms/step


# Applying Adaboost

In [20]:
# AdaBoost implementation
model = AdaBoostClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_, zero_division=0))

# Verify the number of rows processed
print("Number of rows processed:", len(combined_df))


Accuracy: 0.75
              precision    recall  f1-score   support

        fake       0.76      0.82      0.79        57
        real       0.74      0.65      0.69        43

    accuracy                           0.75       100
   macro avg       0.75      0.74      0.74       100
weighted avg       0.75      0.75      0.75       100

Number of rows processed: 500


In [23]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Assuming X_train, X_test, y_train, y_test, and combined_df are already defined

# Define the parameter grid for Grid Search
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0],
    'estimator__max_depth': [1, 2, 3, 4, 5]
}

# Initialize the base estimator
base_estimator = DecisionTreeClassifier(random_state=42)

# Initialize the AdaBoostClassifier with the correct parameter name
model = AdaBoostClassifier(estimator=base_estimator, random_state=42)

# Initialize Grid Search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best estimator
best_model = grid_search.best_estimator_

# Predictions and evaluation
y_pred = best_model.predict(X_test)
print("Best Parameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))

# Verify the number of rows processed
print("Number of rows processed:", len(combined_df))


Best Parameters: {'estimator__max_depth': 4, 'learning_rate': 1.0, 'n_estimators': 200}
Accuracy: 0.85
              precision    recall  f1-score   support

           0       0.84      0.91      0.87        57
           1       0.87      0.77      0.81        43

    accuracy                           0.85       100
   macro avg       0.85      0.84      0.84       100
weighted avg       0.85      0.85      0.85       100

Number of rows processed: 500


> with parameter tuning e achieve accuracy 85.

# Applyng Gboost

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score

# Train and evaluate GBoost
gboost = GradientBoostingClassifier()
gboost.fit(X_train, y_train)
y_pred_gboost = gboost.predict(X_test)
print("Gradient Boosting Classification Report:\n", classification_report(y_test, y_pred_gboost))
print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred_gboost))


Gradient Boosting Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.98      0.88        57
           1       0.97      0.65      0.78        43

    accuracy                           0.84       100
   macro avg       0.88      0.82      0.83       100
weighted avg       0.86      0.84      0.83       100

Gradient Boosting Accuracy: 0.84


In [27]:
# para meter tuning of Gboost 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# Define the parameter grid for Grid Search
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

# Initialize the GradientBoostingClassifier
gboost = GradientBoostingClassifier(random_state=42)

# Initialize Grid Search with cross-validation
grid_search = GridSearchCV(estimator=gboost, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best estimator
best_model = grid_search.best_estimator_

# Predictions and evaluation
y_pred_gboost = best_model.predict(X_test)
print("Best Parameters:", grid_search.best_params_)
print("Gradient Boosting Classification Report:\n", classification_report(y_test, y_pred_gboost, zero_division=0))
print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred_gboost))


/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best Parameters: {'learning_rate': 0.2, 'max_depth': 4, 'max_features': 'log2', 'n_estimators': 200, 'subsample': 0.8}
Gradient Boosting Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.93      0.87        57
           1       0.89      0.72      0.79        43

    accuracy                           0.84       100
   macro avg       0.85      0.83      0.83       100
weighted avg       0.85      0.84      0.84       100

Gradient Boosting Accuracy: 0.84


# Applying XGBoost

In [30]:
# parameter tuning for Xgboost
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# Define the parameter grid for Grid Search
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Initialize the XGBClassifier
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Initialize Grid Search with cross-validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best estimator
best_model = grid_search.best_estimator_

# Predictions and evaluation
y_pred_xgb = best_model.predict(X_test)
print("Best Parameters:", grid_search.best_params_)
print("XGBoost Classification Report:\n", classification_report(y_test, y_pred_xgb, zero_division=0))
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))



Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.89      0.82        57
           1       0.82      0.63      0.71        43

    accuracy                           0.78       100
   macro avg       0.79      0.76      0.77       100
weighted avg       0.79      0.78      0.77       100

XGBoost Accuracy: 0.78


# Applying Neuaral Netwrok with 4 layers 

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from scipy.sparse import csr_matrix
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoding
y_train_encoded = to_categorical(y_train, num_classes=2)
y_test_encoded = to_categorical(y_test, num_classes=2)

# Convert sparse matrices to dense, if necessary
X_train_dense = csr_matrix(X_train).todense()
X_test_dense = csr_matrix(X_test).todense()

# Define the neural network
nn4 = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(2, activation='softmax')  # Use 2 neurons for two classes
])

# Compile the model with appropriate loss function and optimizer
optimizer = Adam(learning_rate=0.001)
nn4.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn4.fit(X_train_dense, y_train_encoded, epochs=60, batch_size=32, verbose=1)

# Predict and evaluate the model
y_pred_nn4 = nn4.predict(X_test_dense)
y_pred_nn4 = (y_pred_nn4 > 0.5).astype("int32")

print("Neural Network (4 layers) Classification Report:\n", classification_report(y_test_encoded, y_pred_nn4))
print("Neural Network (4 layers) Accuracy:", accuracy_score(y_test_encoded.argmax(axis=1), y_pred_nn4.argmax(axis=1)))



Epoch 1/60


I0000 00:00:1717779235.824028   86346 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13/13 [==============================] - 2s 4ms/step - loss: 0.6875 - accuracy: 0.5600
Epoch 2/60
13/13 [==============================] - 0s 4ms/step - loss: 0.6192 - accuracy: 0.6625
Epoch 3/60
13/13 [==============================] - 0s 4ms/step - loss: 0.4233 - accuracy: 0.9425
Epoch 4/60
13/13 [==============================] - 0s 4ms/step - loss: 0.2011 - accuracy: 0.9950
Epoch 5/60
13/13 [==============================] - 0s 4ms/step - loss: 0.0577 - accuracy: 1.0000
Epoch 6/60
13/13 [==============================] - 0s 4ms/step - loss: 0.0095 - accuracy: 1.0000
Epoch 7/60
13/13 [==============================] - 0s 4ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 8/60
13/13 [==============================] - 0s 4ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 9/60
13/13 [==============================] - 0s 4ms/step - loss: 7.5066e-04 - accuracy: 1.0000
Epoch 10/60
13/13 [==============================] - 0s 4ms/step - loss: 5.8657e-04 - accuracy: 1.0000
Epoch 11/60
13/13 [===

# Applying Neural Network with 6 layers

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, accuracy_score
# Convert labels to one-hot encoding
y_train_encoded = to_categorical(y_train, num_classes=2)
y_test_encoded = to_categorical(y_test, num_classes=2)

# Convert sparse matrices to dense, if necessary
X_train_dense = csr_matrix(X_train).todense()
X_test_dense = csr_matrix(X_test).todense()

# Neural network with 6 layers
nn6 = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(2, activation='sigmoid')
])
nn6.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nn6.fit(X_train_dense, y_train_encoded, epochs=50, batch_size=32, verbose=0)
y_pred_nn6 = (nn6.predict(X_test_dense) > 0.5).astype("int32")
print("Neural Network (6 layers) Classification Report:\n", classification_report(y_test_encoded, y_pred_nn6))
print("Neural Network (6 layers) Accuracy:", accuracy_score(y_test_encoded, y_pred_nn6))


4/4 [==============================] - 0s 2ms/step
Neural Network (6 layers) Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.67      0.79        57
           1       0.88      0.84      0.86        43

   micro avg       0.93      0.74      0.82       100
   macro avg       0.93      0.75      0.82       100
weighted avg       0.93      0.74      0.82       100
 samples avg       0.74      0.74      0.74       100

Neural Network (6 layers) Accuracy: 0.74


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
